# Exercise 6 - Linear ML for Econometrics

In this exercise set, we will work with linear ML methods that can give unbiased estimates when the number of covariates is large. We will once again set up simulated data to clearly see any issues with the methods we apply. The exercises follow the approach laid out in [Chernozhukov et al](https://arxiv.org/pdf/1501.03185.pdf).

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import PolynomialFeatures

> The following code simulates two datasets, one to use with the auxilliary regression 
>$$ \tag{aux}
d_i = x_i'\gamma_0 + z_i' \Pi_0 + u_i
$$
>and one to use in the main equation
>$$ \tag{main}
y_i = \alpha_0 d_i + x_i' \delta_0 + \nu_i
$$
Unlike in the paper, we include covariates in the main equation. We will handle them in a later question. In this setup $u_i$ and $\nu_i$ are correlated, meaning the relation between $d_i$ and $y_i$ is not identified via OLS. Instead we will use $z_i$ to induce exogenous variation in $d_i$, which is unrelated to the error terms. This variation can be used to identify $\alpha_0$.
>
> **Ex 6.1.1.**  Rewrite the below code to define a function `simulate(n, m, k, plot)`, where plot is a boolean indicating whether the density plot should be drawn or not. Your function should return all the matrices and vectors required in the regression equations, including parameters and errors.

```
n = 1000  # Number of observations
m = 1500    # Number of potential instruments
k = 10    # Number of covariates

cov = 5
errors = np.random.multivariate_normal(mean = [0,0], cov = [[cov, cov], [cov, cov]], size = n)
h = sns.jointplot(errors[:,0], errors[:,1], kind = 'kde')
h.set_axis_labels('$\\nu$', '$\epsilon$', fontsize=16)

z = np.random.normal(size = (n,m))
x = np.random.normal(size = (n,k))

# Auxilliary equation
nu = errors[:,0]
Pi = np.array([5] + [x if np.random.uniform() > 0.9 else 0 for x in np.random.uniform(low = -2, high = 5, size = m - 1)])
gamma = np.array([5] + [x if np.random.uniform() > 0.9 else 0 for x in np.random.uniform(low = -2, high = 5, size = k - 1)])

d = z @ Pi + x @ gamma + nu

# Main equation
u = errors[:,1]
delta = np.array([5] + [x  if np.random.uniform() > 0.9 else 0 for x in np.random.uniform(low = -2, high = 5, size = k - 1)])
alpha = np.random.uniform(1,2)

y = alpha * d  + x @ delta + u
```

In [ ]:
# [Your solution here]

def simulate(n, m, k, plot = False):
    cov = 5
    errors = np.random.multivariate_normal(mean = [0,0], cov = [[cov, cov], [cov, cov]], size = n)

    z = np.random.normal(size = (n,m))
    x = np.random.normal(size = (n,k))

    # Auxilliary equation
    nu = errors[:,0]
    Pi = np.array([5] + [x if np.random.uniform() > 0.9 else 0 for x in np.random.uniform(low = -2, high = 5, size = m - 1)])
    gamma = np.array([5] + [x if np.random.uniform() > 0.9 else 0 for x in np.random.uniform(low = -2, high = 5, size = k - 1)])

    d = z @ Pi + x @ gamma + nu

    # Main equation
    u = errors[:,1]
    delta = np.array([5] + [x  if np.random.uniform() > 0.9 else 0 for x in 
                            np.random.uniform(low = -2, high = 5, size = k - 1)])
    alpha = np.random.uniform(1,2)

    y = alpha * d  + x @ delta + u
    
    # Make density plot when asked for it
    if plot == True:
        h = sns.jointplot(errors[:,0], errors[:,1], kind = 'kde')
        h.set_axis_labels('$\\nu$', '$\epsilon$', fontsize=16)
        plt.show()
    
    return [y, d, x, z, u, nu, alpha, delta, Pi, gamma]
    

> **Ex. 6.1.2:** Use your function to simulate a new dataset and regress the following OLS regression
>$$
y_i \sim \pi_0 + \pi_1 d_i + \gamma_i
$$
> where $\gamma_i$ is a noise term. 
>
> Repeat this procedure 1000 times in a for loop and store the true $\alpha_0$ as well as the estimate $\pi_1$ in two lists. Plot a histogram of the differences $\alpha_0 - \pi_1$. What does this tell you about the regression you just ran?

**Answer**:

From the historgram plotted below, I see that $\pi_1$ is upward biased, since the difference between $\pi_1$ and the true treatment effect, $\alpha_0$, is always negative. This makes sense, since we're just trying to estimate the treatment effect using OLS with no covariates, but the treatment indicator, $d_i$, is correlated with the error term, $\gamma_i$. 

In [ ]:
import statsmodels.api as sm
from statsmodels.api import OLS
from statsmodels.tools import add_constant

In [ ]:
# [Your solution here]

# Initialise empty lists to fill out in loop
alpha0 = []
pi1 = []

# Simualte data and regress OLS 1000 times
for i in range(1000):
    if i < 1000:
        sim = simulate(n = 1000, m = 1500, k = 10)
        alpha0.append(sim[6])
        
        y = sim[0]
        d = sim[1]
        d = sm.add_constant(d)
        model = sm.OLS(y,d)
        results = model.fit()
        pi1.append(results.params[1])
        
    elif i == 1000:
        break

In [ ]:
# Plot histogram of the difference between the true effect and the biased effect estimated with OLS

# I can't subtract the two because they're lists. So, I have to use a bit complicated method...
diff = []
zip_object = zip(alpha0, pi1)
for list1_i, list2_i in zip_object:
    diff.append(list1_i-list2_i)

plt.hist(diff)
plt.show()

> **Ex. 6.1.3:** Knowing the DGP an obvious solution would be to run an IV regression, instrumenting $d_i$ with $z_i$. Unfortunately there are $m=1500$ columns in $z_i$ and only $n=1000$ observations. Luckily, the way we have simulated our data only a small subset of the $z_i$'s actually influence $d_i$. The tricky part will be to pick out the right $z_i$'s.
>
> To begin with, simulate a new dataset and count the number of non-zero element in $\Pi$ to verify that indeed only very few $z$'s matter for $d$.

**Answer**:

There are 166 non-zero elements in $\Pi$, which is relatively few out of 1500.

In [ ]:
# [Your solution here]
sim_new = simulate(n = 1000, m = 1500, k = 10)
pi = sim_new[8]
print(np.count_nonzero(pi))

> **Ex. 6.1.4:** The _ideal_ instrument for $d_i$ is exactly the $z_i$'s which have non-zero coefficients, multiplied by the corresponding true simulated parameters in $\Pi_0$. Having simulated the data ourselves, an easy way to compute this is to simply calculate
> $$
\hat{d}^* = z \cdot \Pi_0
$$
> where $\cdot$ is the dot product, written as `@` in numpy. In reality we cannot get this ideal instrument, because it would require regressing $d_i$ on all 500 variables with only 100 observations.  
>
> In a for loop over 1000 iterations, simulate new data, compute the ideal instrument $\hat{d_i}$ and regress the second stage regression $y_i \sim \pi_0 + \pi_1\hat{d_i}$. Store the true $\alpha_0$ and the estimate $\hat{\pi}_1$ in two lists. Finally draw a histogram of the differences $\alpha_0 - \hat{\pi}_1$. How does your histogram look this time, is this expected?

**Answer**:

From the histogram plotted below, I see that the estimate of the treatment effect, $\hat{\pi_1}$, is now downward biased, since the difference between the true treatment effect and the estimate is positive. 

In [ ]:
# [Your solution here]

# Initialise lists to fill in loop below
alpha0 = []
p1_hat = []

# Simulate data and regress second stage with ideal instrument 1000 times
for i in range(1000):
    if i < 1000:
        sim_new = simulate(n = 1000, m = 1500, k = 10)
        alpha0.append(sim[6])
        
        # Compute ideal instrument for treatment, d
        z = sim_new[3]
        Pi = sim_new[8]
        d_star = z @ Pi
        
        # Regress the second stage regression
        y = sim[0]
        d = sim[1]
        d_star = sm.add_constant(d_star)
        model = sm.OLS(y, d_star)
        results = model.fit()
        p1_hat.append(results.params[1])
        
    elif i == 1000:
        break


In [ ]:
# Plot histogram of the difference between the true effect and the biased effect estimated with OLS
diff_new = []
zip_object = zip(alpha0, p1_hat)
for list1_i, list2_i in zip_object:
    diff_new.append(list1_i-list2_i)

plt.hist(diff_new)
plt.show()

> **Ex. 6.1.5:** The below class implements post-LASSO. A two-step procedure where first a LASSO model is used to identify relevant parameters, and then OLS is used to estimate parameter values on the remaining variables. Study the code, and understand as well as possible what is going on. 
>
> What is stored in `relevant_x`?
> 
> Why is the `predict` method so complicated?

**Answer**:

- In `relevant_x` is the relevant covariates stored from the LASSO, which is a variable selection and regularization method. Besides the most relevant covariates from the LASSO, `relevant_x` also includes variables forced into the vector, which can be specified in the `force_include_idx` option in the `fit()` function. 

- The `predict` method is complicated because it accounts for three possible cases:
 1. When no covariates/features are specified: It uses `relevant_x` for prediction
 2. When as many covariates are specified in `X` as the number of variables in `relevant_x`: It uses `X` for prediction
 3. When more covariates are specified in `X` than the number of variables in `relevant_x`: It uses the subset of `X` that is in `relevant_x`

 I actually don't understand, why there have to be three different scenarios, since it uses the variables included in `relevant_x` each time. 

In [ ]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [ ]:
class PostLasso:
    def __init__(self, formula = None):
        self.lasso_model = Lasso()
        self.ols_model = None
        self.relevant_x = None
        self.subset_cols = None
        self.coefs = None
        self.formula = formula
    
    def __repr__(self):
        return f'PostLasso({self.formula})'
    
    @ignore_warnings(category=ConvergenceWarning)
    
    def fit(self, X, y, force_include_idx = None):
        ''' Estimate a model using Post-LASSO
        
        X: X matrix (without intercept)
        y: y vector
        force_include_idx: column indexes that ALWAYS is
            included in the OLS model, regardless of their
            status in the LASSO stage.
        '''
        self.lasso_model = self.lasso_model.fit(X,y)
        self.coefs = np.insert(self.lasso_model.coef_, 0, self.lasso_model.intercept_)
        self.subset_cols = np.where(self.coefs != 0)[0]
        
        # This forces some variables to be part of the list of covariates. Typically, you would force the treatment 
        # indicator to be in the covariates
        if force_include_idx is not None:
            self.subset_cols = np.union1d(self.subset_cols, force_include_idx)
        self.relevant_x = add_constant(X)[:,self.subset_cols]
        self.ols_model = OLS(y, self.relevant_x).fit()
        return self

    def predict(self, X = None):
        ''' Predict using a fitted post-lasso model.
        '''
        if X is None:
            return self.ols_model.predict(self.relevant_x)
        if X.shape == self.relevant_x.shape:
            return self.ols_model.predict(X)
        return self.ols_model.predict(X[:,self.subset_cols])

> **Ex. 6.1.6:** In this problem we will try to run through the post-lasso steps required to obtain an estimate of $\alpha_0$. Since we are doing this in python we will not be able to set the lasso hyperparameter optimally for this kind of post-selection usage. There is a R package, developed especially to handle inference after lasso-selection, which you should use in practise. 
>
> For now, do the following steps 1000 times, storing the true $\alpha_0$ and estimate $\hat{\alpha_0}$:
>
> * 0. Simulate a new dataset.
> * 1. Run a post-lasso regression of d on x and z, $d_i \sim x_i'\gamma + z_i' \delta$, forcing the inclusion of $x_i$ in the OLS stage.
> * 2. Run the second stage regression $y_i \sim \hat{d}_i + x_i' \beta$ to recover $\hat{\alpha_0}$.
>
> How does this histogram compare to the naive one? How does it compare to the ideal one?
>
> _Hint:_ We follow the description given on page 19 [here](https://cran.r-project.org/web/packages/hdm/vignettes/hdm.pdf).

**Answer**:

I've tried to do the above, but I think my histogram looks weird. I thought I would find that the post-LASSO method resulted in an unbiased estimate of the treatment effect, but it looks like the estimate is biased. From the histogram, the estimate should be downwards biased and the bias/difference from the true treatment effect from the simulations are uniformly distributed from around 1.1 to 2.1. 

In [ ]:
# [Your solution here]
# Remove warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Initialise lists to fill in loop below
alpha0 = []
alpha0_hat = []

for i in range(1000):
    if i < 1000:
        # 0. Simulate 
        sim_new = simulate(n = 1000, m = 1500, k = 10)
        alpha0.append(sim_new[6])

        # 1. Run post-LASSO regrssion of d on X and z
        # I have to concatenate X and z, since fit() only takes inputs X and y
        X = np.concatenate((sim_new[2], sim_new[3]), axis=1)
        d = sim_new[1]
        force_include_idx = range(10)

        post_lasso = PostLasso()
        fit = post_lasso.fit(X, d, force_include_idx)
        d_hat = post_lasso.predict()
        
        # 2. Run second stage regression of y on d and X
        y = sim_new[0]
        d_hat = sm.add_constant(d_hat) # Should I add constant??
        
        second_stage = sm.OLS(y, d_hat, X)
        res = model.fit()
        alpha0_hat.append(res.params[0])
        
    elif i == 1000:
        break

In [ ]:
# Plot histogram of difference between true and estimated treatment effect
diff_new = []
zip_object = zip(alpha0, alpha0_hat)
for list1_i, list2_i in zip_object:
    diff_new.append(list1_i-list2_i)

plt.hist(diff_new)
plt.show()